In [85]:
import geopandas as gpd
import math
from shapely.geometry import Point, LineString, Polygon, MultiPoint, MultiLineString, MultiPolygon
import pyproj

In [86]:
buildings= gpd.read_file('../tests/data/buildings.geojson')
road= gpd.read_file('../tests/data/road.geojson')

In [87]:
# for d in buildings.columns:
#     print(d)
road['end'] = None
road['start'] = None
road['start_point'] = None
road['end_point'] = None
road['fwd_azimuth'] = None
road['back_azimuth'] = None
buildings['road_direction'] = None
buildings['road_length_d_code'] = None

print(buildings)
print(road)

      OBJECTID_1     NEW                             geometry road_direction  \
0              1   TS108  POINT Z (83.46114 27.56682 0.00000)           None   
1              2   TS108  POINT Z (83.46127 27.56692 0.00000)           None   
2              3   TS108  POINT Z (83.46143 27.56676 0.00000)           None   
3              4   TW974  POINT Z (83.45739 27.56961 0.00000)           None   
4              5   TW974  POINT Z (83.45731 27.56982 0.00000)           None   
...          ...     ...                                  ...            ...   
6540        6541  TS1342  POINT Z (83.47218 27.60221 0.00000)           None   
6541        6542   TS835  POINT Z (83.46734 27.60720 0.00000)           None   
6542        6543  TS1510  POINT Z (83.46667 27.60676 0.00000)           None   
6543        6544   TS839  POINT Z (83.46641 27.60665 0.00000)           None   
6544        6545   TS202  POINT Z (83.46638 27.60718 0.00000)           None   

     road_length_d_code  
0            

In [132]:
class bdcPoint:
    	
	def __init__(self):
		
		self.x = 0
		self.y = 0

# Constant integers for directions
RIGHT = 1
LEFT = -1
ZERO = 0

def directionOfPoint(A, B, P):
    """[Provides direction of the point in respect to line]

    Args:
        A ([type]): [Starting Point of Line]
        B ([type]): [Ending Point of Line]
        P ([type]): [Point of Interest]

    Returns:
        [Direction(int)]: [Left,Right and Zero : -1,1,0 ( Left direction, Right Direction , On Line)]
    """
    global RIGHT, LEFT, ZERO
	
	# Subtracting co-ordinates of
	# point A from B and P, to
	# make A as origin
 
    B.x -= A.x
    B.y -= A.y
    P.x -= A.x
    P.y -= A.y

    # Determining cross Product
    """"The Cross-Product has an interesting Property which will be used to determine direction of a point from a line segment. That is, the cross-product of two points is positive if and only if the angle of those point at origin (0, 0) is in counter-clockwise. And conversely the cross-product is negative if and only if the angle of those point at origin is in clockwise direction."""
    cross_product = B.x * P.y - B.y * P.x

    # Return RIGHT if cross product is positive
    if (cross_product > 0):
        return RIGHT
        
    # Return LEFT if cross product is negative
    if (cross_product < 0):
        return LEFT

    # Return ZERO if cross product is zero
    return ZERO

def addDirectionLength(direction,length,index):
    #right-even, left-odd number 
    length=int(length)
    print(length)
    if (direction == 1):
        print("Right Direction")
        buildings.at[index,'road_direction']="right"
        buildings.at[index,'road_length_d_code']=round_up_to_even(length)
        
    elif (direction == -1):
        print("Left Direction")
        buildings.at[index,'road_direction']="left"
        buildings.at[index,'road_length_d_code']=round_up_to_odd(length)
    else:
        buildings.at[index,'road_direction']="on_line"
        
def getStartEndPoint(line,type):
    if len(line)==1 or type=="road" :
       
        df_line = line.geometry.boundary
        # print(len(df_line))
        # print(line_bound)
        # df_line=line_bound.explode(index_parts=True)
        start_point,end_point=df_line[0],df_line[1]
        # projected_line = line.to_crs(epsg=32644)
        # length= projected_line.geometry.length
        # float_length=length.tolist()
        return start_point,end_point
    else:
        raise ValueError ("Line contains more than one code")


def round_up_to_even(f):
    return math.ceil(f / 2.) * 2

def round_up_to_odd(f):
    return math.ceil(f) // 2 * 2 + 1

def addStartEndPoint(start_point,end_point,index):
    
    road.at[index,'start_point']=start_point
    road.at[index,'end_point']=end_point
    

def FindIntersectedFeatures(start_point,end_point,line_row,index_road):
    # projected_line = line_row.to_crs(epsg=32644)
    
    # buffer=projected_line.buffer(0.2)
    # print(buffer)
    # projected_road=road.to_crs(epsg=32644) 
    for index, row in road.iterrows():
        line=row.geometry
        if str(row.NEW) != str(line_row.NEW):
            if start_point.within(line):
                print("start found "+ str(row.NEW))
                road.at[index_road,'start']=str(row.NEW)
            if end_point.within(line):
                print("end found "+ str(row.NEW))
                road.at[index_road,'end']=str(row.NEW)

def CalculateTouchLine():
    for index,row in road.iterrows():
        # print(row)
        start_point,end_point=getStartEndPoint(row,"road")
        addStartEndPoint(start_point,end_point,index)
        FindIntersectedFeatures(start_point,end_point,row,index)
    print("Sucessfully added : startpoint, endpoint , Start touching Line , End Touching Line")

def CalculateBearing(start_point,end_point,index):
    geodesic = pyproj.Geod(ellps='WGS84')
    fwd_azimuth,back_azimuth,distance = geodesic.inv(start_point.x, start_point.y, end_point.x, end_point.y)
    road.at[index,'fwd_azimuth']=fwd_azimuth
    road.at[index,'back_azimuth']=back_azimuth
    
                   


In [133]:
CalculateTouchLine()
# print(road.loc[0])

/tmp/ipykernel_3905/963436991.py:73: ShapelyDeprecationWarning: __getitem__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  start_point,end_point=df_line[0],df_line[1]


end found TC075
start found TS923
end found TC075
end found TC075
end found TC075
end found TC075
end found TC075
end found TC075
end found TC075
start found TS881
end found TC075
end found TC075
end found TC075
start found TS1107
end found TC075
end found TC075
end found TC075
end found TC075
end found TC075
start found TS1366
end found TC075
end found TC075
end found TC075
end found TC075
end found TC075
start found TS110
end found TC075
end found TC075
start found TS881
end found TC075
start found TS881
end found TC075
end found TC075
end found TC075
end found TC075
end found TC075
end found TC075
end found TC075
end found TC075
end found TC075
end found TC075
end found TC075
end found TC075
start found TS959
end found TC075
end found TC075
end found TC075
start found TS062
end found TC075
end found TC075
end found TC075
end found TC075
end found TC075
start found TS973
end found TS1366
end found TS062
end found TS020
start found TS1107
end found TS1366
start found TS863
end found T

In [ ]:
for index, row in buildings.iterrows():
    print (index)
    print (row.NEW)
    road_code = row.NEW
    poi = row.geometry
    road_linked_wrt_point= road.loc[road['NEW'] == road_code]
    try:
        start_point,end_point,length=getStartEndLength(road_linked_wrt_point)
    except:
        raise ValueError(row.NEW)
        break
    
    FindIntersectedFeatures(start_point,end_point,road_linked_wrt_point,int.from_bytes(road_linked_wrt_point.index,"little"))
    addStartEndPoint(start_point,end_point,int.from_bytes(road_linked_wrt_point.index,"little"))
    
    # start = bdcPoint()
    # end = bdcPoint()
    # point = bdcPoint()
    # start.x,start.y,end.x,end.y,point.x,point.y=start_point.x,start_point.y,end_point.x,end_point.y,poi.x,poi.y
    # direction=directionOfPoint(start,end,point)
    # addDirectionLength(direction,length,index)
    # print(row)
# print (road.loc[5])

In [134]:
road.to_file("../app_outputs/output.geojson", driver="GeoJSON")

ValueError: Invalid field type <class 'shapely.geometry.point.Point'>

In [139]:
print(road.loc[800])

OBJECTID                                                     801
OBJECTID_1                                                   411
Code                                                       D3677
Road_Nam_1                                                  None
OBJECTID_2                                                   393
Shape_Le_2                                         188.605888562
Ward_No                                                     13.0
Surface                                              Blacktopped
Priority_1                                                  None
Priority                                                     0.0
Width_1                                                     None
ROW_1                                                       None
Width                                                       24.0
Class                                                          D
ROW                                                         20.0
NEW                      